Поскольку в датасете очень мало положительных примеров, размножим этот класс при помощи перефразирования

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
import pandas as pd
import torch
from google.colab import drive
from transformers import T5ForConditionalGeneration, T5Tokenizer
from tqdm import tqdm

In [17]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
drive.mount('/content/drive/')
train_data = pd.read_csv('/content/drive/MyDrive/Sber/train_data.csv', index_col=0)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [18]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
MODEL_NAME = 'cointegrated/rut5-base-paraphraser'
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
model = model.to(device)
tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model.eval();

def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    x = x.to(device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [11]:
text = train_data.iloc[1]['sentence']
print(paraphrase(text, grams=3, do_sample=False))

В 2014-2018 гг. Генпрокуратура РФ выявила 17 тыс. различных правонарушений – хищения бюджета, задержания сроков постройки, невыплаты заработной платы и прочих фактов, произошедших при строительстве космоса.


In [12]:
train_data['label'].value_counts()

0    1340
1     329
Name: label, dtype: int64

In [19]:
train_data_1 = train_data.head(329)

In [20]:
%%time
paraphrases = []
for i in tqdm(range(len(train_data_1))):
  paraphrases.append(paraphrase(train_data_1.iloc[i]['sentence']))

100%|██████████| 329/329 [04:56<00:00,  1.11it/s]

CPU times: user 4min 52s, sys: 1.57 s, total: 4min 54s
Wall time: 4min 56s


In [21]:
train_data_1.insert(1, 'paraphrases', paraphrases)
train_data_1.drop('sentence', axis=1, inplace=True)
train_data_1.rename(columns={'paraphrases': 'sentence'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
train_data_augmented = train_data.append(train_data_1)

In [24]:
train_data_augmented['label'].value_counts()

0    1340
1     658
Name: label, dtype: int64

In [23]:
train_data_augmented.to_csv('train_data_augmented.csv')